In [0]:
from pyspark.sql.functions import col,monotonically_increasing_id
from delta.tables import DeltaTable

### **Parameters**


In [0]:
catalog="flights"                                                                       #catalog_name

source_schema="silver"                                                                  #source schema
source_object="silver_flights"                                                          #source table

target_schema="gold"                                                                    #target schema
target_object="dim_flights"                                                             #target table

cdc_col="modified_date"                                                                 #cdc column name

surrogate_key="dims_flights_key"                                                        #surrogate key column name

key_columns="['flight_id']"                                                             #key columns name
key_col_list=eval(key_columns)                                                          #key columns list

backdated_refresh=""                                                                    #backdate fill value


### **Incremental Ingestion**

In [0]:
#Compute Last load date
last_load_date=backdated_refresh

if len(backdated_refresh)==0:
    if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
        last_load_date=spark.sql(f"""
                                 SELECT max({cdc_col}) From {catalog}.{target_schema}.{target_object}
                                 """).collect()[0][0]
    else:
        last_load_date="1900-01-01 00:00:00"

#checking last_load_date
(last_load_date)


In [0]:
#fetching incremental source data
#change >= to > if not done already

df_src=spark.sql(f"""
                 SELECT * FROM {catalog}.{source_schema}.{source_object} WHERE {cdc_col}>='{last_load_date}'    
                 """)
df_src.display()


In [0]:
#creating key_col_list for incremental and initial table

if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    key_col_incremental=', '.join([f"{col}" for col in key_col_list])
    df_tgt=spark.sql(f"""
                     Select {key_col_string},{surrogate_key},create_date,update_date 
                     from {catalog}.{target_schema}.{target_object}
                     """)
else:
    key_col_initial=', '.join([f"'' as {col}" for col in key_col_list])
    df_tgt=spark.sql(f"""
                     Select {key_col_initial},'' as {surrogate_key},'1900-01-01 00:00:00' as create_date,'1900-01-01 00:00:00' as update_date 
                     where 1=0
                     """)

#displaying target df
df_tgt.display()    
    
